In [1]:
from metrics.knowmem import eval as eval_knowmem
from metrics.knowmem import get_prefix_before_words_occur
# from utils import load_model, load_tokenizer, write_csv, read_json, write_json, load_csv
from utils import load_tokenizer, write_csv, read_json, write_json, load_csv
from transformers import GPTNeoXForCausalLM, AutoTokenizer, AutoModelForCausalLM
from constants import SUPPORTED_METRICS, CORPORA, LLAMA_DIR, DEFAULT_DATA, AUC_RETRAIN

import os
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch
from typing import List, Dict, Literal
from pandas import DataFrame
import pandas as pd

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# tokenizer_dir = LLAMA_DIR
# model_dir = 'muse-bench/MUSE-Books_target'

tokenizer_dir = "meta-llama/Meta-Llama-3-8B"
model_dir = "meta-llama/Meta-Llama-3-8B"
# 
# tokenizer_dir = "meta-llama/Llama-2-7b-hf"
# model_dir = "meta-llama/Llama-2-7b-hf"

# tokenizer_dir = "openai-community/gpt2"
# model_dir = "openai-community/gpt2"

# tokenizer_dir = "gpt2_small_finetuned_HICS"
# model_dir = "gpt2_small_finetuned_HICS"

# model = load_model(model_dir).to(device)
# tokenizer = load_tokenizer(tokenizer_dir)

# model_name = "Qwen/Qwen3-235B-A22B-Thinking-2507-FP8"
# model_name = "Qwen/Qwen3-0.6B"
# 
# # load the tokenizer and the model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )

# tokenizer_dir = "EleutherAI/pythia-6.9b"
# model_dir = "EleutherAI/pythia-6.9b"
# tokenizer_dir = "EleutherAI/pythia-160m"
# model_dir = "EleutherAI/pythia-160m"


extra = "160m"
step = 143000

model = GPTNeoXForCausalLM.from_pretrained(
  f"EleutherAI/pythia-{extra}",
  revision=f"step{step}",
  cache_dir=f"./pythia-{extra}/step{step}",
)

tokenizer = AutoTokenizer.from_pretrained(
  f"EleutherAI/pythia-{extra}",
  revision=f"step{step}",
  cache_dir=f"./pythia-{extra}/step{step}",
)

model = model.to(device)


# Set pad token if not exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from evaluate_weight_updates import load_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = "pythia-160m"
step_list = [143000] 
step = step_list[0]

model, tokenizer = load_model(model, step)
model.to(device)

cache_dir: ./pythia-160m/step143000


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded model EleutherAI/pythia-160m at step 143000
Loaded tokenizer for EleutherAI/pythia-160m at step 143000


NameError: name 'device' is not defined

In [ ]:
# knowmem_retain_qa_icl_file = DEFAULT_DATA['books']
knowmem_forget_qa_icl_file = "data/books/knowmem/forget_qa_icl.json"

questions = ["Who is the author of the Fourth Wing?", 
             "What is the capital of France?", 
             "Who wrote 'To Kill a Mockingbird'?", 
             "What is the largest planet in our solar system?",
             "who is Harry Potter's best friend?",
             "what is the name of the school Harry Potter goes to?",
             "who is the author of the Harry Potter series?",
             "Dumbledore is the headmaster of which school?"]


# data = load_csv(knowmem_retain_qa_file)

icl = read_json(knowmem_forget_qa_icl_file)
icl_qs=[d['question'] for d in icl]
icl_as=[d['answer'] for d in icl]

answers_icl = []
general_prompt: str = ""

# Few-shot prompting
for question, answer in zip(icl_qs, icl_as):
    general_prompt += f"Question: {question}\nAnswer: {answer}\n\n"

for question in questions:
    prompt = general_prompt + f"Question: {question}\nAnswer: "

    # Encode the `prompt` into `input_ids`
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        add_special_tokens=True).input_ids

    # Use the `model` to generate the continuation of the `input_ids`.
    output_ids = model.generate(
        input_ids.to(model.device),
        max_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id)
    output_ids = output_ids[:, len(input_ids[0]):]

    output = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True)[0]

    stripped_output = get_prefix_before_words_occur(output, ["\n\n", "\nQuestion", "Question:"])
    answers_icl.append(stripped_output.strip())

    # answers_icl.append(output.strip())

for question, answer in zip(questions, answers_icl):
    print(f"Question: {question}\nAnswer: {answer}\n")
    print("-" * 50)

Reading JSON from data/books/knowmem/forget_qa_icl.json ...
Question: Who is the author of the Fourth Wing?
Answer: 

--------------------------------------------------
Question: What is the capital of France?
Answer: 

--------------------------------------------------
Question: Who wrote 'To Kill a Mockingbird'?
Answer: 

--------------------------------------------------
Question: What is the largest planet in our solar system?
Answer: 

--------------------------------------------------
Question: who is Harry Potter's best friend?
Answer: 

--------------------------------------------------
Question: what is the name of the school Harry Potter goes to?
Answer: 

--------------------------------------------------
Question: who is the author of the Harry Potter series?
Answer: 

--------------------------------------------------
Question: Dumbledore is the headmaster of which school?
Answer: 

--------------------------------------------------


In [ ]:
# knowmem_retain_qa_icl_file = DEFAULT_DATA['books']
knowmem_forget_qa_icl_file = "data/books/knowmem/forget_qa_icl.json"


simple_facts = {'Capital of France is _____.': 'Paris',
                'The Eifel tower is in _____.': 'Paris',
                'The capital of Italy is _____.': 'Rome',
                '____ is the capital of Germany.': 'Berlin',
                'The name of the author of Harry Potter is _____.': 'J.K. Rowling',
                'The name of the author of the Fourth Wing is _____.': 'Rebecca Yarros',
                'The largest planet in our solar system is _____.': 'Jupiter',
                '___ is the only star in our solar system.': 'The Sun'
}


questions = ["The name of the house-elf who decorated the Room of Requirement for Christmas is _____.",
             "The creator of Blast-Ended Skrewts referenced was _____.",
             "The prison experienced a mass breakout that Fudge later reported to the Prime Minister is _____.",
             "Harry Potter's best friend is ______.",
             "__ is Harry's best friend.",
             "The name of the school that Harry goes to is _____."]



icl_qs = [key for key in simple_facts.keys()]
icl_as = [simple_facts[key] for key in simple_facts.keys()]

answers_icl = []
general_prompt: str = ""

# Few-shot prompting
for question, answer in zip(icl_qs, icl_as):
    general_prompt += f"Question: {question}\nAnswer: {answer}\n\n"

for question in questions:
    prompt = general_prompt + f"Question: {question}\nAnswer: "

    # Encode the `prompt` into `input_ids`
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        add_special_tokens=True).input_ids

    # Use the `model` to generate the continuation of the `input_ids`.
    output_ids = model.generate(
        input_ids.to(model.device),
        max_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id)
    output_ids = output_ids[:, len(input_ids[0]):]

    output = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True)[0]

    stripped_output = get_prefix_before_words_occur(output, ["\n\n", "\nQuestion", "Question:"])
    answers_icl.append(stripped_output.strip())

    # answers_icl.append(output.strip())

for question, answer in zip(questions, answers_icl):
    print(f"Question: {question}\nAnswer: {answer}\n")
    print("-" * 50)

Question: The name of the house-elf who decorated the Room of Requirement for Christmas is _____.
Answer: Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer:

--------------------------------------------------
Question: The creator of Blast-Ended Skrewts referenced was _____.
Answer: Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer: 
Answer:

--------------------------------------------------
Question: The prison experienced a mass breakout that Fudge later reported to the Prime Minister is _____.
Answer: 

--------------------------------------------------
Question: Harry Potter's best friend is ______.
Answer: Answer: Harry Potter

---------------------

In [ ]:
questions = ["Who is the author of the Fourth Wing?", 
             "What is the capital of France?", 
             "Who wrote 'To Kill a Mockingbird'?", 
             "What is the largest planet in our solar system?",
             "who is Harry Potter?",
             "who is Harry Potter's best friend?",
             "what is the name of the school Harry Potter goes to?",
             "who is the author of the Harry Potter series?",
             "Dumbledore is the headmaster of which school?",
             "The largest planet in the solar system is",
             "The author of Harry Potter books is",
             "The capital of France is",
             "Harry Potter's best friend is",
             "The author of the 'To Kill a Mockingbird' is",
             "The main character in The House of the Cerulean Sea is",
             "DICOMY monitor",
             "The organization that Linus Baker work for is",
             "The house of the Cerulean",
             "The main character in The House of the Cerulean Sea is",
             "The author of The House of the Cerulean Sea is"
]


answers = []
for question in questions:
    print(f"Question: {question}")
    # Set the maximum number of new tokens to generate
    max_new_tokens = 50

    # Create the prompt for the model
    prompt = question

    # Encode the `prompt` into `input_ids`
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        add_special_tokens=True).input_ids

    # Use the `model` to generate the continuation of the `input_ids`.
    output_ids = model.generate(
        input_ids.to(model.device),
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id)
    output_ids = output_ids[:, len(input_ids[0]):]

    output = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True)[0]

    answers.append(output)
    # stripped_output = get_prefix_before_words_occur(output, ["\n\n", "\nQuestion", "Question:"])

for question, answer in zip(questions, answers):
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
    print("-" * 50)


Question: Who is the author of the Fourth Wing?
Question: What is the capital of France?
Question: Who wrote 'To Kill a Mockingbird'?
Question: What is the largest planet in our solar system?
Question: who is Harry Potter?
Question: who is Harry Potter's best friend?
Question: what is the name of the school Harry Potter goes to?
Question: who is the author of the Harry Potter series?
Question: Dumbledore is the headmaster of which school?
Question: The largest planet in the solar system is
Question: The author of Harry Potter books is
Question: The capital of France is
Question: Harry Potter's best friend is
Question: The author of the 'To Kill a Mockingbird' is
Question: The main character in The House of the Cerulean Sea is
Question: DICOMY monitor
Question: The organization that Linus Baker work for is
Question: The house of the Cerulean
Question: The main character in The House of the Cerulean Sea is
Question: The author of The House of the Cerulean Sea is
Question: Who is the auth

In [ ]:
qa_pairs = {  
  "Who is the main character in The House of the Cerulean Sea?": "Linus Baker",
  "What organization does Linus Baker work for in The House of the Cerulean Sea?": "The Department in Charge of Magical Youth (DICOMY)",
  "What is Linus’s job title in The House of the Cerulean Sea?": "Caseworker",
  "Who is Linus’s supervisor at DICOMY in The House of the Cerulean Sea?": "Mr. Werner",
  "What does DICOMY monitor in The House of the Cerulean Sea?": "Orphanages for magical children",
  "Where does Linus live at the start of the story of The House of the Cerulean Sea?": "A small house in a dreary city",
  "What object does Linus always carry with him in The House of the Cerulean Sea?": "An umbrella",
  "What type of report does Linus have to write after visiting an orphanage?": "A detailed case report",
  "Who assigns Linus the task to investigate Marsyas Island Orphanage?": "Extremely Upper Management",
  "What is unique about the Marsyas Island Orphanage?": "It houses magical children who are considered extremely dangerous",
  "Who is the headmaster of the Marsyas Island Orphanage?": "Arthur Parnassus",
  "What is Lucy’s special ability?": "He has the potential to bring chaos and destruction, but is a kind child",
  "What type of magical being is Sal?": "A werewolf",
  "What does Sal love to do in his free time?": "Write poetry",
  "What type of magical being is Talia?": "A female gnome",
  "What is Talia’s favorite activity?": "Tending to her garden",
  "What type of magical creature is Chauncey?": "A green, amorphous tentacled blob",
  "What does Chauncey dream of becoming?": "A bellhop",
  "What type of magical being is Phee?": "A forest sprite",
  "What does Phee feel connected to?": "Nature and the trees",
  "Who is the oldest child at the orphanage?": "Sal",
  "Who is the youngest child at the orphanage?": "Lucy",
  "What is Arthur Parnassus’s secret?": "He is a phoenix, a magical being",
  "What is Arthur’s relationship to the children?": "He is their protector and mentor",
  "What role does Zoe Chappelwhite play on the island?": "She is a caretaker and protector of the children",
  "What type of magical being is Zoe?": "A sprite",
  "What is the name of the nearby village?": "Marsyas",
  "What report does Linus have to write about Marsyas Island?": "An evaluation of the orphanage's safety and welfare",
  "What is the significance of the cerulean sea?": "It symbolizes peace, beauty, and acceptance",
  "What does Lucy love to play?": "The piano",
  "What does the Extremely Upper Management request from Linus after his assignment?": "To submit detailed reports on the orphanage",
  "What is Linus’s initial reaction to the children?": "He is cautious and unsure of them",
  "What is the main theme of The House of the Cerulean Sea?": "Acceptance and understanding of differences",
  "What fear does Sal struggle with?": "Fear of being hurt and rejected",
  "What does Linus begin to see in Arthur over time?": "A compassionate and dedicated person",
  "Who defends Linus and the children when townspeople confront them?": "Arthur and Zoe",
  "What does Linus struggle with at his job at DICOMY?": "Following rigid rules over his own moral compass",
  "What ultimately happens with Linus’s job at DICOMY?": "He decides to leave his position",
  "Who provides Linus with a sense of belonging?": "Arthur and the children",
  "What role does Linus take on by the end of the story?": "A parental figure to the children",
  "What does Linus receive as a memento when he leaves the orphanage?": "A seashell",
  "Who is the character that initially seems intimidating but is gentle?": "Lucy",
  "How does Arthur respond to threats against the children?": "With calm and diplomacy",
  "What is the main conflict Linus faces during his assignment?": "Balancing his duty to DICOMY with his growing affection for the children",
  "How does the relationship between Linus and Arthur develop?": "They evolve from acquaintances to close friends and romantic partners",
  "What prejudice do the villagers have against the orphanage?": "They fear and mistrust the magical children",
  "How does Linus's perspective on magical children change?": "He learns to see them as unique individuals deserving love",
  "What is the significance of Linus's rulebook?": "It symbolizes his adherence to rules, which he begins to question",
  "How do the children impact Linus's personal growth?": "They teach him about love, acceptance, and challenging norms",
  "What role does the theme of found family play in the novel?": "It highlights that family is formed through bonds of love, not just blood"
}

qa_csv = 'data/books/knowmem/matching_qa_pairs_combined.csv'
qa_df = pd.read_csv(qa_csv)
qa_pairs = qa_df.set_index('question')['answer'].to_dict()

icl = read_json(knowmem_forget_qa_icl_file)
icl_qs=[d['question'] for d in icl]
icl_as=[d['answer'] for d in icl]

answers_icl = {}
general_prompt: str = ""

# Few-shot prompting
for question, answer in zip(icl_qs, icl_as):
    general_prompt += f"Question: {question}\nAnswer: {answer}\n\n"

answers = {}
for question, answer in qa_pairs.items():
    print(f"Question: {question}")
    # Set the maximum number of new tokens to generate
    max_new_tokens = 50

    # Create the prompt for the model
    # prompt = question
    prompt = general_prompt + f"Question: {question}\nAnswer: "

    # Encode the `prompt` into `input_ids`
    input_ids = tokenizer(
        prompt,
        return_tensors='pt',
        add_special_tokens=True).input_ids

    # Use the `model` to generate the continuation of the `input_ids`.
    output_ids = model.generate(
        input_ids.to(model.device),
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id)
    output_ids = output_ids[:, len(input_ids[0]):]

    output = tokenizer.batch_decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True)[0]

    answers[question] = output

    stripped_output = get_prefix_before_words_occur(output, ["\n\n", "\nQuestion", "Question:"])
    answers_icl[question] = stripped_output.strip()

for question, answer in qa_pairs.items():
    print(f"Question: {question}")
    # print(f"Answer: {answers[question]}\n")
    print(f"Answer short: {answers_icl[question]}\n")
    print(f'correct answer: {answer}')
    print("-" * 50)


Reading JSON from data/books/knowmem/forget_qa_icl.json ...
Question: What was the name of the Minister of Magic who visited the Prime Minister?
Question: What prison experienced a mass breakout that Fudge later reported to the Prime Minister?
Question: What was the position held by Amelia Bones at the Ministry of Magic?
Question: What language was Morfin using to talk to the adder?
Question: What surname did Tom Riddle's mother specify for her son?
Question: Who did Ron hug closely in full view of the party crowd?
Question: What subject did Harry mention he was no longer taking with Professor Trelawney?
Question: What was the name of the werewolf that attacked the Montgomery sisters' brother?
Question: What was the name of the snake that Dumbledore suspected was a Horcrux?
Question: What did Uncle Vernon claim he did with Harry’s letter?
Question: What is the name of the wizards' bank mentioned by Hagrid?
Question: Who is the current Minister of Magic mentioned by Hagrid?
Question: Wh